In [ ]:
# =======================================================================
# ✅  مخطط Sankey بناءً على تقسيم Leiden (4 مجموعات)
# =======================================================================
#@title  **Sankey Diagram of Community Allocation**

# --- [الخطوة 0: التثبيت] ---
print("--- [Step 0: Installing Libraries] ---")
!pip install -qq plotly seaborn networkx pandas numpy python-igraph leidenalg
print("Libraries installed successfully.\n")
!git clone https://github.com/NoorBayan/Amthal.git
%cd Amthal
from utility import *
from IPython.display import display, HTML, clear_output

# --- [الخطوة 1: الاستيراد] ---
print("--- [Step 1: Importing Libraries] ---")
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import networkx as nx
import seaborn as sns
from itertools import combinations
import igraph as ig
import leidenalg as la
print("Libraries imported successfully.\n")


# --- [الخطوة 2: تحميل البيانات] ---
print("--- [Step 2: Loading Data] ---")
INSTANCES_FILE_PATH = '/content/Amthal/data/processed/instances.csv'
try:
    df = pd.read_csv(INSTANCES_FILE_PATH, sep='\t', encoding='utf-16')
    df['Dominant_Concept']=[Dominant_Concept_dic[i]['en'] for i in df.Dominant_Concept]
    df['Valence']=[Valence_dic[i]['ar'] for i in df.Valence]
    df['Revelation_Phase']=[Revelation_Phase_dic[i]['ar'] for i in df.Revelation_Phase]
    df['Abstraction_Level']=[Abstraction_Level_dic[i]['ar'] for i in df.Abstraction_Level]

    print(f"Data loaded successfully. Found {len(df)} instances.\n")
except FileNotFoundError:
    print(f"⚠️ ERROR: File not found at '{INSTANCES_FILE_PATH}'. Please check the path and try again.")
    exit()


# --- [الخطوة 3: بناء الشبكة الدقيقة واكتشاف المجموعات باستخدام Leiden] ---
# (هذه الخطوة ضرورية للحصول على التقسيم الصحيح المكون من 4 مجموعات)
print("--- [Step 3: Building Network and Detecting Communities with Leiden] ---")
G_nx = nx.Graph()
verses = df.groupby(['Sura_No', 'Aya_No'])
for name, group in verses:
    concepts_in_verse = list(set(group['Dominant_Concept']))
    if len(concepts_in_verse) > 1:
        verse_text = group['vers_text'].iloc[0]
        verse_length = len(str(verse_text).split())
        weight = 1 / verse_length if verse_length > 0 else 0
        for concept1, concept2 in combinations(concepts_in_verse, 2):
            if G_nx.has_edge(concept1, concept2):
                G_nx[concept1][concept2]['weight'] += weight
            else:
                G_nx.add_edge(concept1, concept2, weight=weight)

G_ig = ig.Graph.from_networkx(G_nx)
partition_leiden = la.find_partition(G_ig, la.RBConfigurationVertexPartition, weights='weight', seed=42)

# إنشاء قاموس للتقسيم (مفهوم -> رقم المجموعة)
partition_dict = {}
for i, community in enumerate(partition_leiden):
    for vertex_index in community:
        node_name = G_ig.vs[vertex_index]['_nx_name']
        partition_dict[node_name] = i

partition_df = pd.DataFrame(list(partition_dict.items()), columns=['Dominant_Concept', 'Community_ID'])
print(f"Network partitioned successfully into {len(set(partition_dict.values()))} communities.\n")


# --- [الخطوة 4: تجميع البيانات وتعيين التسميات الجديدة للمجموعات الأربع] ---
print("--- [Step 4: Aggregating Data and Assigning New Labels] ---")
instance_counts = df['Dominant_Concept'].value_counts().reset_index()
instance_counts.columns = ['Dominant_Concept', 'Instance_Count']

sankey_data = pd.merge(instance_counts, partition_df, on='Dominant_Concept')

# ✅✅✅  تحديث: استخدام التسميات الجديدة والصحيحة للمجموعات الأربع
# تأكد من أن هذه التسميات تتوافق مع الترتيب والتفسير في النص الرئيسي
community_labels_new = {
    0: 'The Sphere of Social and Moral Action',
    1: 'The Ideological Kernel',
    2: 'Cosmological and Symbolic Forces',
    3: 'The Dynamics of Spiritual Existence'
}
# ملاحظة: قد تحتاج لتعديل الأرقام (0, 1, 2, 3) لتطابق المجموعات الفعلية التي ينتجها الكود.
# يمكنك طباعة `partition_df` للتحقق من أي رقم يخص أي مجموعة.
sankey_data['Community_Label'] = sankey_data['Community_ID'].map(community_labels_new)
print("New community labels for 4 communities assigned.\n")


# --- [الخطوة 5: تجهيز البيانات لمخطط Sankey] ---
print("--- [Step 5: Preparing Data for Plotly Sankey] ---")
sankey_data = sankey_data.sort_values(by='Instance_Count', ascending=False)
all_labels = pd.concat([sankey_data['Dominant_Concept'], sankey_data['Community_Label']]).unique().tolist()
source_indices = [all_labels.index(concept) for concept in sankey_data['Dominant_Concept']]
target_indices = [all_labels.index(label) for label in sankey_data['Community_Label']]
values = sankey_data['Instance_Count']
print("Sankey data prepared.\n")


# --- [الخطوة 6: إنشاء المخطط بتنسيق أكاديمي وألوان للمجموعات الأربع] ---
print("--- [Step 6: Generating Academic-Style Sankey Diagram] ---")

# 🎨 استخدام لوحة ألوان هادئة للمجموعات الأربع
num_communities = len(community_labels_new)
community_colors_palette = sns.color_palette("muted", num_communities)
community_colors_rgba = [f'rgba({int(r*255)}, {int(g*255)}, {int(b*255)}, 0.85)' for r, g, b in community_colors_palette]

# ✅ تحديث: تعيين لون فريد لكل مجتمع من المجتمعات الأربع
community_color_map = {label: color for label, color in zip(community_labels_new.values(), community_colors_rgba)}

# إنشاء ألوان العقد
node_colors = ['rgba(210, 210, 210, 0.9)' if label not in community_color_map else community_color_map[label] for label in all_labels]

# إنشاء ألوان الروابط
link_colors = [community_color_map[sankey_data.iloc[i]['Community_Label']].replace('0.85', '0.35') for i in range(len(sankey_data))]

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="rgba(80,80,80,0.6)", width=0.6),
        label=all_labels,
        color=node_colors
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values,
        color=link_colors
    )
)])

# --- [إعدادات الشكل النهائية للنشر] ---
fig.update_layout(
    title_text="Distribution of Conceptual Instances into Four Thematic Communities",
    font=dict(size=12, family="Times New Roman", color="rgba(40,40,40,1)"),
    title_font=dict(size=15, family="Times New Roman", color="rgba(20,20,20,1)"),
    title_x=0.5,
    width=500, # زيادة العرض قليلاً لاستيعاب التسميات الأطول
    height=650,
    margin=dict(l=10, r=10, t=50, b=10),
    paper_bgcolor='white',
    plot_bgcolor='white'
)

fig.show()

# لحفظ الشكل، ستحتاج لمكتبة kaleido
# !pip install -qq kaleido
# fig.write_image("Figure_4b_Revised_Sankey.png", scale=3) # scale=3 للحصول على دقة عالية

print("✅ Academic-style Sankey diagram generated successfully.")

--- [Step 0: Installing Libraries] ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 63.6 MB/s eta 0:00:00
Libraries installed successfully.

--- [Step 1: Importing Libraries] ---
Libraries imported successfully.

--- [Step 2: Loading Data] ---
Data loaded successfully. Found 4078 instances.

--- [Step 3: Building Network and Detecting Communities with Leiden] ---
Network partitioned successfully into 4 communities.

--- [Step 4: Aggregating Data and Assigning New Labels] ---
New community labels for 4 communities assigned.

--- [Step 5: Preparing Data for Plotly Sankey] ---
Sankey data prepared.

--- [Step 6: Generating Academic-Style Sankey Diagram] ---


✅ Academic-style Sankey diagram generated successfully.
